In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.constants import Boltzmann as kB 
from tkinter import *

# Parameters
m = 1
v = 10
N_part = 625
m_disk = 10
R_disk = 10
L = 260
sigma = 1
epsilon = 1
T_tot = 10
dt = 0.005
trajectory = []
msd_list = []



#  visualization setup
window_size = 600
scaling_factor = window_size / L 

tk = Tk()
tk.geometry(f'{window_size + 20}x{window_size + 20}')
tk.configure(background='#000000')
tk.title("Brownian Simulation")

canvas = Canvas(tk, background='#ECECEC')
canvas.place(x=10, y=10, height=window_size, width=window_size)

# Initialize disk
disk = {'x': L / 2, 'y': L / 2, 'vx': 0, 'vy': 0}
disk_obj = canvas.create_oval(
    (disk['x'] - R_disk) * scaling_factor + window_size / 2,
    (disk['y'] - R_disk) * scaling_factor + window_size / 2,
    (disk['x'] + R_disk) * scaling_factor + window_size / 2,
    (disk['y'] + R_disk) * scaling_factor + window_size / 2,
    fill="black"
)


# Particle initial positions
x = [L / 2 + i % int(L / sigma) * sigma for i in range(N_part)]
y = [L / 2 + i // int(L / sigma) * sigma for i in range(N_part)]

particles = []
particle_objs = []
for i in range(int(np.sqrt(N_part))):
    for j in range(int(np.sqrt(N_part))):
        x = i * L / np.sqrt(N_part) + L / (2 * np.sqrt(N_part))
        y = j * L / np.sqrt(N_part) + L / (2 * np.sqrt(N_part))
        angle = np.random.uniform(0, 2 * np.pi)
        vx = v * np.cos(angle)
        vy = v * np.sin(angle)
        particle = {'x': x, 'y': y, 'vx': vx, 'vy': vy}
        
        # Skip particles within disk radius + 3*sigma
        if np.sqrt((particle['x'] - disk['x']) ** 2 + (particle['y'] - disk['y']) ** 2) > (R_disk + 3 * sigma):
            particles.append(particle)
            particle_obj = canvas.create_oval(
                (x - sigma / 2) * scaling_factor + window_size / 2, 
                (y - sigma / 2) * scaling_factor + window_size / 2,
                (x + sigma / 2) * scaling_factor + window_size / 2, 
                (y + sigma / 2) * scaling_factor + window_size / 2,
                outline='#00C0C0', 
                fill='#00C0C0'
            )
            particle_objs.append(particle_obj)

# Stop simulation on Escape key press
def stop_loop(event):
    global running
    running = False

tk.bind("<Escape>", stop_loop)




# Remove particles within the disk radius plus 3*sigma
particles = [p for p in particles if np.sqrt((p['x'] - disk['x']) ** 2 + (p['y'] - disk['y']) ** 2) > (R_disk + 3 * sigma)]




In [2]:
# Plot trajectory and MSD
def plot_results():
    # Plot disk's trajectory
    trajectory_x, trajectory_y = zip(*trajectory)
    plt.figure(figsize=(6, 5))  # 创建单独的图窗口
    plt.plot(trajectory_x, trajectory_y, color='blue')
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.title("Disk Trajectory in Cartesian Plane")
    plt.grid(True)
    plt.savefig( 'disk_trajectory.png') 
    plt.close() 

    # MSD Plot
    plt.figure(figsize=(6, 5)) 
    time = np.arange(1, len(msd_list) + 1) * dt
    plt.plot(time, msd_list, color='red')
    plt.xlabel("Time (t)")
    plt.ylabel("Mean Square Displacement (MSD)")
    plt.title("Mean Square Displacement over Time")
    plt.grid(True)
    plt.savefig('msd_over_time.png')
    plt.close()

# Lennard-Jones force calculation
def lennard_jones_force(disk, particle):
    dx = disk['x'] - particle['x']
    dy = disk['y'] - particle['y']
    r = np.sqrt(dx ** 2 + dy ** 2) - R_disk
    if r < 3 * sigma:
        F = 24 * epsilon * ((2 * (sigma / r) ** 12) - (sigma / r) ** 6) / r
        fx = F * dx / r
        fy = F * dy / r
        return fx, fy
    return 0, 0


trajectory = []
msd_list = []
time_steps = int(T_tot / dt)


# Main loop with Tkinter refresh
step = 0
running = True

while running and step < time_steps:
    # Record disk's position for trajectory and MSD calculation
    trajectory.append((disk['x'], disk['y']))

    # Calculate MSD at each step
    if step > 0:
        initial_x, initial_y = trajectory[0]
        msd = (disk['x'] - initial_x) ** 2 + (disk['y'] - initial_y) ** 2
        msd_list.append(msd)

    # Update disk position
    disk_fx, disk_fy = 0, 0
    for i, particle in enumerate(particles):
        fx, fy = lennard_jones_force(disk, particle)
        disk_fx += fx
        disk_fy += fy
        particle['vx'] += -fx / m * dt
        particle['vy'] += -fy / m * dt
        particle['x'] += particle['vx'] * dt
        particle['y'] += particle['vy'] * dt

        # Boundary conditions for particles (reflecting)
        if particle['x'] <= 0 or particle['x'] >= L:
            particle['vx'] *= -1
        if particle['y'] <= 0 or particle['y'] >= L:
            particle['vy'] *= -1

        # Update particle position in Tkinter
        canvas.coords(particle_objs[i], 
                      (particle['x'] - sigma / 2) * scaling_factor + window_size / 2,
                      (particle['y'] - sigma / 2) * scaling_factor + window_size / 2,
                      (particle['x'] + sigma / 2) * scaling_factor + window_size / 2,
                      (particle['y'] + sigma / 2) * scaling_factor + window_size / 2)

    # Update disk velocity and position based on net force
    disk['vx'] += disk_fx / m_disk * dt
    disk['vy'] += disk_fy / m_disk * dt
    disk['x'] += disk['vx'] * dt
    disk['y'] += disk['vy'] * dt

    # Boundary conditions for the disk (reflecting)
    if disk['x'] <= R_disk or disk['x'] >= L - R_disk:
        disk['vx'] *= -1
    if disk['y'] <= R_disk or disk['y'] >= L - R_disk:
        disk['vy'] *= -1

    # Update disk position in Tkinter
    canvas.coords(disk_obj, 
                  (disk['x'] - R_disk) * scaling_factor + window_size / 2,
                  (disk['y'] - R_disk) * scaling_factor + window_size / 2,
                  (disk['x'] + R_disk) * scaling_factor + window_size / 2,
                  (disk['y'] + R_disk) * scaling_factor + window_size / 2)

    

    if step % 100 == 0:
        print(f'Time step: {step}, Disk position: ({disk["x"]}, {disk["y"]})')

    # Update Tkinter display
    tk.update_idletasks()
    tk.update()
    time.sleep(0.01) 

    # Increment step
    step += 1

# End of simulation
plot_results()
tk.mainloop()




Time step: 0, Disk position: (130.0, 130.0)
Time step: 100, Disk position: (130.15794463268276, 130.14179599862933)
Time step: 200, Disk position: (130.82007722112635, 130.73664400912062)
Time step: 300, Disk position: (131.4821161220252, 131.33164784062518)
Time step: 400, Disk position: (132.13602381283403, 131.94132020968573)
Time step: 500, Disk position: (132.84134626361887, 132.08793099606498)


2024-11-13 08:44:31.947 Python[2341:54379] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-13 08:44:31.947 Python[2341:54379] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Time step: 600, Disk position: (133.7110622935401, 131.82229216547026)
Time step: 700, Disk position: (134.7364949627278, 131.54090460270092)
Time step: 800, Disk position: (135.754826201431, 131.28337662128484)
Time step: 900, Disk position: (136.23546368166174, 131.14130312297831)
Time step: 1000, Disk position: (136.45425656319554, 131.74303968736956)


TclError: invalid command name ".!canvas"